In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, tree, model_selection, preprocessing, pipeline

# Регрессия

In [2]:
def cross_validation_result(estimator):
    # Проводим кросс-валидацию с разбиением на 3 отрезка
    cv = model_selection.cross_validate(estimator, X, y, scoring=scores, cv=3)
    # Выводим средние результаты
    print('R2 score: ', cv['test_r2'].mean())
    print('MSE score: ', cv['test_neg_mean_squared_error'].mean())
    print('RMSE score: ', cv['test_neg_root_mean_squared_error'].mean())
    print('MAE score: ', cv['test_neg_mean_absolute_error'].mean())


def grid_search_result(estimator, param_grid):
    # Подбираем гиперпараметры с помощью GridSearchCV
    grid = model_selection.GridSearchCV(estimator, param_grid, cv=5)
    grid.fit(X, y)
    cross_validation_result(grid)


### Подготовка данных

In [3]:
# Считываем данные из файла и перемешиваем, так как автомобили отсортированы по году выпуска
mpg = pd.read_csv("../data/auto_mpg_preprocessed.csv")
mpg = mpg.drop(columns="Unnamed: 0").sample(frac=1)

In [4]:
# Отделяем ключевой атрибут и наименования автомобилей 
X = mpg.drop(columns=['mpg','car name']).values
y = mpg['mpg'].values

In [5]:
# Объявляем критерии оценки
scores = ['r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']
# Объявляем скейлер
scaler = preprocessing.StandardScaler()

### Линейная регрессия

In [6]:
%%time
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
LR = linear_model.LinearRegression()
lr_pipe = pipeline.Pipeline(steps=[('scaler', scaler), ('LR', LR)])

cross_validation_result(lr_pipe)

R2 score:  0.8075550024019429
MSE score:  -11.706689430886927
RMSE score:  -3.413425372783937
MAE score:  -2.579890747190856
Wall time: 73 ms


У линейной регрессии нет гиперпараметров, поэтому подбор не проводится

### Регрессия дерева решений

In [7]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
tree_regressor = tree.DecisionTreeRegressor()
tree_pipe = pipeline.Pipeline(steps=[('scaler', scaler), ('tree', tree_regressor)])
# Проводим кросс-валидацию
cross_validation_result(tree_pipe)

R2 score:  0.7657971991768332
MSE score:  -14.201442748091603
RMSE score:  -3.765976083833509
MAE score:  -2.58153650420826


#### Подбор гиперпараметров для дерева решений
Гиперпараметры дерева решений:
- criterion - функция определения качества разделения вершин
- splitter - стратегия разделения вершин
- max_depth - максимальная глубина дерева
- min_samples_leaf - минимальное кол-во объектов в листе

In [8]:
%%time
# Подбираем гиперпараметры с помощью GridSearchCV
tree_param_grid = dict(tree__criterion=['mse', 'friedman_mse', 'mae', 'poisson'], tree__splitter=['best', 'random'], tree__min_samples_leaf=[1, 3, 5, 10, 15], tree__max_depth=[5, 10, 20, 50, 100, None])
grid_search_result(tree_pipe, tree_param_grid)

R2 score:  0.8101816946906668
MSE score:  -11.555856710338531
RMSE score:  -3.389340548703587
MAE score:  -2.481186895660178
Wall time: 13.3 s


### Lasso регрессия

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
lasso = linear_model.Lasso()
lasso_pipe = pipeline.Pipeline(steps=[('scaler', scaler), ('lasso', lasso)])
# Проводим кросс-валидацию
cross_validation_result(lasso_pipe)

R2 score:  0.7819229757307783
MSE score:  -13.314137607126193
RMSE score:  -3.6308815977161273
MAE score:  -2.691703402504104


#### Подбор гиперпараметров для Lasso регрессии
Гиперпараметры lasso:
alpha - штрафующий коэффициент

In [10]:
%%time
# Подбираем гиперпараметры с помощью GridSearchCV
lasso_param_grid = dict(lasso__alpha=np.arange(0.05, 1, 0.05))
grid_search_result(lasso_pipe, lasso_param_grid)

R2 score:  0.8083610958897847
MSE score:  -11.654838118668346
RMSE score:  -3.4062523436482057
MAE score:  -2.5575843983237427
Wall time: 810 ms


### Гребневая регрессия

In [11]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
ridge = linear_model.Ridge()
ridge_pipe = pipeline.Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
# Проводим кросс-валидацию
cross_validation_result(ridge_pipe)

R2 score:  0.8074526319461599
MSE score:  -11.710897830177094
RMSE score:  -3.414083752618535
MAE score:  -2.574096949792262


#### Подбор гиперпараметров для Ridge регрессии
Гиперпараметры ridge:
alpha - штрафующий коэффициент

In [12]:
%%time
# Подбираем гиперпараметры с помощью GridSearchCV
ridge_param_grid = dict(ridge__alpha=np.arange(0.05, 1, 0.05))
grid_search_result(ridge_pipe, ridge_param_grid)


R2 score:  0.8068423171862809
MSE score:  -11.75000195032248
RMSE score:  -3.419581214035569
MAE score:  -2.580346566834231
Wall time: 775 ms


### Elastic Net регрессия

In [13]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
elastic = linear_model.ElasticNet()
elastic_pipe = pipeline.Pipeline(steps=[('scaler', scaler), ('elastic', elastic)])
# Проводим кросс-валидацию
cross_validation_result(elastic_pipe)

R2 score:  0.7558261380534725
MSE score:  -14.888049729952314
RMSE score:  -3.843104591213738
MAE score:  -2.859218025849581


#### Подбор гиперпараметров для Elastic Net регрессии
Гиперпараметры Elastic Net:
alpha - штрафующий коэффициент
l1_ratio - коэффициент смешивания, т.е. l1_ratio = 0 будет означать использование только l2 регуляризации, а l1_ratio - только l1

In [14]:
%%time
# Подбираем гиперпараметры с помощью GridSearchCV
elastic_param_grid = dict(elastic__alpha=np.arange(0.05, 1, 0.05), elastic__l1_ratio=np.arange(0.1, 0.9, 0.05))
grid_search_result(elastic_pipe, elastic_param_grid)

R2 score:  0.806100960573309
MSE score:  -11.791620333723763
RMSE score:  -3.425500150162344
MAE score:  -2.5650942646962016
Wall time: 13 s


### Выводы
В целом, все модели после подбора параметров выдали практически одинаковые результаты. <br>
Скорее всего, это связано с тем, что зависимость ключевого атрибута от большинства параметров является близкой к линейной. <br>
Также, стоит заметить, что для получения таких результатов данные пришлось перемешать, так как они были отсортированы по году выпуска